In [ ]:
# %pip install langchain==0.0.309

In [ ]:
import langchain
langchain.__version__

In [ ]:
import json
import os
import sys
import boto3
import botocore
from langchain.chains import LLMChain
from langchain.llms.bedrock import Bedrock
from langchain.prompts import PromptTemplate
from langchain.embeddings import BedrockEmbeddings

In [ ]:
boto3_bedrock = boto3.client(service_name='bedrock-runtime')

body = json.dumps({
    "prompt": "\n\nHuman:explain black holes to 8th graders\n\nAssistant:",
    "max_tokens_to_sample": 300,
    "temperature": 0.1,
    "top_p": 0.9,
})

modelId = 'anthropic.claude-v2'
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

response_body = json.loads(response.get('body').read())
# text
print(response_body.get('completion'))

In [ ]:
#boto3_bedrock.list_foundation_models()

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from langchain.chains import create_extraction_chain
from langchain.llms import Bedrock


llm = Bedrock(
    region_name="us-east-1",
    client=boto3_bedrock,
    model_id="anthropic.claude-v2",
    model_kwargs={"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 500}
)
print(llm("what is SageMaker?"))

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.llms.bedrock import Bedrock
#from utils import bedrock, print_ww


import warnings
warnings.filterwarnings('ignore')
# turn verbose to true to see the full logs and documents
conversation= ConversationChain(prompt=claude_prompt,
    llm=llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

# langchain prompts do not always work with all the models. This prompt is tuned for Claude
claude_prompt = PromptTemplate.from_template("""

Human: You are an AI assistant for an Airline called WorldAir. You are helpful, polite friendly. Start by asking how you can help.
The users are not allowed to ask about things outside of  questions relating to booking or traveling.  It is very important that you finish collecting the following fields from the user:
The user's full name that includes first name and family name.
The user's departure location
The user's destination location
One way or return
The user's dates of travel, check if the dates are flexible or non negotiable
number of people traveling, ask both for adults and children under 12 years old
The user's budget for the trip
Any additional preferences or constraints the user may have


Current conversation:
<conversation_history>
{history}
</conversation_history>

Here is the human's next reply:
<human_reply>
{input}
</human_reply>

Assistant:
""")

#conversation.prompt = claude_prompt

print(conversation.predict(input="Hi there!"))


In [ ]:
print(conversation.predict(input="i'd like to book a flight, can you help me please?"))

In [ ]:
print(conversation.predict(input="Romina sharifpour and i am travelling from Melbourne to Auckland"))

# Collect the history of all the conversation and feed it into another chain to give back slots

In [ ]:
history= conversation.memory.buffer
print(history)

In [ ]:
from langchain.prompts import PromptTemplate

slots_prompt = PromptTemplate.from_template(
    """From all the provided input, you should then extract and write EODC (for end of data collection) and on a new line output all these fields as JSON. "user_name": "<user_name>", "departure_location": <departure_location>, "destination_location": "<destination_location>", "dates_of_travel": "<dates_of_travel>", "budget": <budget>, "additional_preferences": "<additional_preferences>.
<input>
{input}
</input>   
Assistant:
"""
)

runnable = slots_prompt | llm | StrOutputParser()
output=runnable.invoke({"input": history})
print(output)